# Google Places API
----

Geocoding requires billing
https://developers.google.com/maps/documentation/geocoding/usage-and-billing
Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../openweather/cities.csv')
df



,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1,dunedin,100,NZ,1603573340,56,-45.87,170.50,64.65,11.10
1,2,sinnamary,41,GF,1603573340,74,5.38,-52.95,83.17,12.57
2,3,puerto ayora,40,EC,1603573340,73,-0.74,-90.35,77.00,14.99
3,4,hermanus,6,ZA,1603573340,91,-34.42,19.23,62.01,4.00
4,5,busselton,71,AU,1603573341,50,-33.65,115.33,57.00,10.00
...,...,...,...,...,...,...,...,...,...,...
551,599,iracemapolis,75,BR,1603573259,61,-22.58,-47.52,78.80,3.36
552,601,chunhuhub,79,MX,1603573402,88,19.55,-88.68,84.38,3.85
553,602,yerbogachen,100,RU,1603573402,97,61.28,108.01,23.97,3.13
554,603,rawlins,90,US,1603573402,42,41.79,-107.24,46.40,23.04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Needed to ENABLE Geocoding API for heatmaps
# See https://stackoverflow.com/questions/32994634/this-api-project-is-not-authorized-to-use-this-api-please-ensure-that-this-api
gmaps.configure(api_key=g_key) 
locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

<img src="fig-1.png" />

### Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.

In [4]:
ideal = df[(df['Max Temp'] > 70) & (df['Max Temp'] < 72)]


### Hotel Map

In [5]:
# Drop any rows will null values.
# Restart index numbering, drop prevents old index from being inserted into the dataframe
# This is important because the next cell uses DataFrame.at to set the hotel name
hotel_df = ideal.dropna().reset_index(drop=True) 

# Add a "Hotel Name" column to the DataFrame.
hotel_df.insert(1, "Hotel Name", None)
hotel_df



,City ID,Hotel Name,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,None,saint-philippe,0,RE,1603573197,64,-21.36,55.77,71.60,5.82
1,8,None,mweka,43,CD,1603573341,96,-4.85,21.57,70.59,0.67
2,18,None,mataura,93,NZ,1603573342,58,-46.19,168.86,71.01,15.99
3,43,None,arraial do cabo,90,BR,1603573205,94,-22.97,-42.03,71.60,14.99
4,53,None,kindu,100,CD,1603573345,95,-2.95,25.95,71.82,2.24
5,97,None,yangambi,65,CD,1603573349,95,0.81,24.43,71.69,1.66
6,132,None,bluff,85,NZ,1603573216,58,-46.60,168.33,71.01,15.99
7,136,None,masumbwe,0,TZ,1603573353,69,-3.63,32.18,70.95,3.33
8,155,None,rangapara,91,IN,1603573354,95,26.82,92.65,70.93,6.08
9,219,None,santa maria do suacui,100,BR,1603573361,88,-18.19,-42.41,70.72,1.63


In [6]:
# Call Google Places API to add first Hotel name to each city
# See https://developers.google.com/places/web-service/search#nearby-search-and-text-search-responses

# index = 0
for index in range(0,hotel_df['City'].count()):
    city = hotel_df.iloc[index]['City']
    lat = hotel_df.iloc[index]['Lat']
    lng = hotel_df.iloc[index]['Lng']

    print(f'Processing Record {index} | {city} {lat}, {lng}')
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=hotel&key={g_key}'
    response = requests.get(url)
    if response.status_code == 200:
        # response.json() Keys are 'html_attributions', 'next_page_token', 'results', 'status'
        # print(response.json().get('results')[0]) # Returns a dict
        # response.json().get('results') Keys are 'geometry', 'icon', 'name', 'photos', 'place_id', 'reference', 'scope', 'types', 'vicinity'

        # Look at each hotel name. The first name is never what seems like a hotel name, for some reason. 
        # Suggest using the second hotel and not the first, as the readme says
        # for result in response.json().get('results'):
        #     print(result.get('name'))
        which_hotel = 1 # Readme says use 0

        hotel_name = response.json().get('results')[which_hotel].get('name')
        hotel_df.at[index, 'Hotel Name'] = hotel_name

hotel_df

Processing Record 0 | saint-philippe -21.36, 55.77
Processing Record 1 | mweka -4.85, 21.57
Processing Record 2 | mataura -46.19, 168.86
Processing Record 3 | arraial do cabo -22.97, -42.03
Processing Record 4 | kindu -2.95, 25.95
Processing Record 5 | yangambi 0.81, 24.43
Processing Record 6 | bluff -46.6, 168.33
Processing Record 7 | masumbwe -3.63, 32.18
Processing Record 8 | rangapara 26.82, 92.65
Processing Record 9 | santa maria do suacui -18.19, -42.41
Processing Record 10 | richards bay -28.78, 32.04
Processing Record 11 | alexandria 31.22, 29.96
Processing Record 12 | jind 29.32, 76.32
Processing Record 13 | nouadhibou 20.93, -17.03
Processing Record 14 | telde 27.99, -15.42
Processing Record 15 | kampene -3.6, 26.67
Processing Record 16 | sita road 27.03, 67.85
Processing Record 17 | tuatapere -46.13, 167.68
Processing Record 18 | nosy varika -20.58, 48.53
Processing Record 19 | naze 28.37, 129.48
Processing Record 20 | otane -39.88, 176.63
Processing Record 21 | yeppoon -23.

,City ID,Hotel Name,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,"Chambres d'hôte ""La Trinité""",saint-philippe,0,RE,1603573197,64,-21.36,55.77,71.60,5.82
1,8,Cathedral of St. Martin,mweka,43,CD,1603573341,96,-4.85,21.57,70.59,0.67
2,18,Ellie's Villa,mataura,93,NZ,1603573342,58,-46.19,168.86,71.01,15.99
3,43,Pousada Porto Praia,arraial do cabo,90,BR,1603573205,94,-22.97,-42.03,71.60,14.99
4,53,socearuco,kindu,100,CD,1603573345,95,-2.95,25.95,71.82,2.24
5,97,Yangambi Reserve,yangambi,65,CD,1603573349,95,0.81,24.43,71.69,1.66
6,132,Bluff Homestead - Guesthouse & Campervan Park,bluff,85,NZ,1603573216,58,-46.60,168.33,71.01,15.99
7,136,Seventh Day Adventist Church MASUMBWE,masumbwe,0,TZ,1603573353,69,-3.63,32.18,70.95,3.33
8,155,State Bank of India ATM,rangapara,91,IN,1603573354,95,26.82,92.65,70.93,6.08
9,219,Eletrozema,santa maria do suacui,100,BR,1603573361,88,-18.19,-42.41,70.72,1.63


In [7]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<img src="fig-2.png" />